In [132]:
def probAttr(data,attr,val):
    Total=data.shape[0]
    cnt = len(data[data[attr] == val])
    return cnt,cnt/Total

In [133]:
def train(data,AttributeList,concept):
    Attr={}
    probability_list={}
    #Get attribute values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
    
    conceptVals = list(set(data[concept]))
    conceptProbs = {}
    countConcept={}
    AttrConcept = {}
    for cVal in conceptVals:
        countConcept[cVal],conceptProbs[cVal] = probAttr(data,concept,cVal)
    
    for val in Attr:
        probability_list[val]={}
        AttrConcept[val] = {}
        for v in Attr[val]:
            AttrConcept[val][v]={}
            a,probability_list[val][v]=probAttr(data,val,v)
            for cVal in conceptVals:
                dataTemp = data[data[val]==v]
                AttrConcept[val][v][cVal] = len(dataTemp[dataTemp[concept] == cVal])/countConcept[cVal]
            
    print("P(A) : ",conceptProbs,"\n")
    print("P(X/A) : ",AttrConcept,"\n")
    print("P(X) : ",probability_list,"\n")
    return conceptProbs,AttrConcept,probability_list

In [134]:
def test(examples,AttributeList,conceptProbs,AttrConcept,probability_list,data,concept_list,Total):
    misclassification_count=0
    Total1 = len(examples)
    i=1
    for ex in examples:
        px={}
        for x in ex:
            for a in AttributeList:
                for c in concept_list:
                    if x in AttrConcept[a]:
                        if c not in px:
                            px[c] = conceptProbs[c]*AttrConcept[a][x][c]/probability_list[a][x]
                        else:
                            px[c] = px[c]*AttrConcept[a][x][c]/probability_list[a][x]
        
        print(px)
        classification = max(px,key=px.get)
        print(i,"Classification :",classification,"Expected :",ex[-1])
        i=i+1
        if(classification!=ex[-1]):
            misclassification_count+=1
    misclassification_rate=misclassification_count*100/Total1
    accuracy=100-misclassification_rate
    print("Misclassification Count={}".format(misclassification_count))
    print("Misclassification Rate={}%".format(misclassification_rate))
    print("Accuracy={}%".format(accuracy))

In [135]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('data_train1.csv')
    AttributeList=list(data)[:-1]
    concept=str(list(data)[-1])
    Total=data.shape[0]
    conceptProbs,AttrConcept,probability_list = train(data,AttributeList,concept)

    examples = DataFrame.from_csv('data_test1.csv')
    concept_list = list(set(data[concept]))
    test(examples.values,AttributeList,conceptProbs,AttrConcept,probability_list,data,concept_list,Total)

In [136]:
main()

P(A) :  {'Yes': 0.6428571428571429, 'No': 0.35714285714285715} 

P(X/A) :  {'Outlook': {'Rain': {'Yes': 0.3333333333333333, 'No': 0.4}, 'Sunny': {'Yes': 0.2222222222222222, 'No': 0.6}, 'Overcast': {'Yes': 0.4444444444444444, 'No': 0.0}}, 'Temperature': {'Hot': {'Yes': 0.2222222222222222, 'No': 0.4}, 'Mild': {'Yes': 0.4444444444444444, 'No': 0.4}, 'Cool': {'Yes': 0.3333333333333333, 'No': 0.2}}, 'Humidity': {'High': {'Yes': 0.3333333333333333, 'No': 0.8}, 'Normal': {'Yes': 0.6666666666666666, 'No': 0.2}}, 'Wind': {'Strong': {'Yes': 0.3333333333333333, 'No': 0.6}, 'Weak': {'Yes': 0.6666666666666666, 'No': 0.4}}} 

P(X) :  {'Outlook': {'Rain': 0.35714285714285715, 'Sunny': 0.35714285714285715, 'Overcast': 0.2857142857142857}, 'Temperature': {'Hot': 0.2857142857142857, 'Mild': 0.42857142857142855, 'Cool': 0.2857142857142857}, 'Humidity': {'High': 0.5, 'Normal': 0.5}, 'Wind': {'Strong': 0.42857142857142855, 'Weak': 0.5714285714285714}} 

{'Yes': 0.24197530864197522, 'No': 0.9408000000000001